# 1. Importing Libraries and mounting Google Drive

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 7.1MB/s 
     |████████████████████████████████| 890kB 20.2MB/s 
     |████████████████████████████████| 2.9MB 38.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=69d829c9ad39878989243f9aab1979581a5229ca4b3af34ae1c9625e943d0ef6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# 2. Load Training and Development Data Paths

In [ ]:
path = "/content/drive/My Drive/datasets/submission_datasets/"
### Change this to the 

In [ ]:
path_training = path + "train_rev_mcl.csv"
path_mcl_en_rev = path_training
ausemcor_supersence_path = path +  "ausemcor_supersense_pruned.csv"
wic_train_new_format_path =  path + "wic_train_new_format.csv"
dev_en_rev_path = path + "dev_en_rev.csv"
devfinal_en_path =  path + "devfinal-en.csv"
devmix_200_rev_path = path + "dev_mix_200_rev.csv"
path_train_mcl = path + "train_mcl.csv"
path_mix6k_train_rev = path + "mix6k_train_rev"

path_save_3_1 = "/content/drive/MyDrive/datasets/sub/models/English/xlmr_mcl_rev"
path_save_3_2 = "/content/drive/MyDrive/datasets/sub/models/English/roberta_signal_mcl_rev_2"
path_save_3_3 = "/content/drive/MyDrive/datasets/sub/models/English/roberta_signal_aug_rev"
path_save_3_4 = "/content/drive/MyDrive/datasets/sub/models/English/bert_large_mcl_rev_dev"

path_mix_4_1 = "/content/drive/MyDrive/datasets/sub/models/Mix/mbert_mix_no_en_1"
path_mix_4_2 = "/content/drive/MyDrive/datasets/sub/models/Mix/mbert_mix_no_en_no_revmcl_1"
path_mix_4_3 = "/content/drive/MyDrive/datasets/sub/models/Mix/mbert_mix_no_en_no_revmcl_xlwic_1"
path_mix_4_4 = "/content/drive/MyDrive/datasets/final/models/Mix/xlmr_mix_no_en_5"
path_mix_4_5 = "/content/drive/MyDrive/datasets/sub/models/Mix/xlmr_mix_no_en_wic_xlwic"
path_mix_4_6 = "/content/drive/MyDrive/datasets/sub/models/Mix/xlmr_mix_no_en_rev"


farsi_df_path = path + "farsi_200_rev.csv"
xl_zh_df_path = path + "xlwic_zh_val_deepak_format.csv"
xl_nl_df_path = path + "xlwic_dutch_nl_val_deepak_format.csv"
xl_hr_df_path = path + "xlwic_croation_hr_val_deepak_format.csv"
xl_da_df_path = path + "xlwic_danish_da_val_deepak_format.csv"

# **3. Training Models - English- English**

## 3.1 XLM-R MCL-rev

In [ ]:
train_df = pd.read_csv(path_mcl_en_rev)
train_df.columns = ['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag','id']
train_df = train_df[['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag']]



In [ ]:
dev_df = pd.read_csv(dev_en_rev_path)

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



from transformers import XLMRobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-large')


# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []

list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])

for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)

print('signal added ...')


encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)


encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)

train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

print('Wordpiece indices ....')

train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)


dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )



from transformers import XLMRobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "xlm-roberta-large"
        hidden_states = False
        self.encoder = XLMRobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer


class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)



optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )


def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)


def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


path_this = path_save_3_1

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 10
best_val_accuracy = 0
t_best_val_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # if epoch_i ==0:
        #   break

        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step == len(train_dataloader)/2:
          print(" Step Half")
          print("")
          print("Running Validation...")

          t0 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model_bert.eval()
          model_log_reg.eval()

          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch in dev_dataloader:

              with torch.no_grad():
                  b_input_ids = batch[0].to(device)
                  b_attention_mask = batch[1].to(device)
                  b_poses = batch[2].to(device)
                  b_labels = batch[3].to(device)        

                  last_layer = model_bert(b_input_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              # print(logits)
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          # print(all_dev_logits)
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
          print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
          
          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(dev_dataloader)
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)

          if(avg_val_accuracy > best_val_accuracy):
            flag = True
            best_val_accuracy = avg_val_accuracy
            best_val_preds = all_dev_logits
            np.save(os.path.join(path_this, "dev_preds"), best_val_preds)
            torch.save(model_bert.state_dict(), os.path.join(path_this, "roberta_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_this, "log_reg_check_best_dev"))
          
          print("  Validation Loss: {0:.3f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))


          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch in train_dataloader:
              
              with torch.no_grad():
                  b_input_ids = batch[0].to(device)
                  b_attention_mask = batch[1].to(device)
                  b_poses = batch[2].to(device)
                  b_labels = batch[3].to(device)        

                  last_layer = model_bert(b_input_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
          print("Train Accuracy: {0:.4f}".format(avg_val_accuracy))

          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(train_dataloader)
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)
          
          print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
          print("  Train Inference took: {:}".format(validation_time))

          model_bert.train()
          model_log_reg.train()

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        optimizer.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        last_layer = model_bert(b_input_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:

        with torch.no_grad():
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_poses = batch[2].to(device)
            b_labels = batch[3].to(device)        

            last_layer = model_bert(b_input_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      flag = True
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_this, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_this, "log_reg_check_best_dev"))
      np.save(os.path.join(path_this, "best_dev_preds"), best_val_preds)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in train_dataloader:
        
        with torch.no_grad():
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_poses = batch[2].to(device)
            b_labels = batch[3].to(device)        

            last_layer = model_bert(b_input_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    print("Train Accuracy: {0:.4f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
    print("  Training Inference took: {:}".format(validation_time))

    # t_labels = []
    # t_preds = []

    # # Evaluate data for one epoch
    # for batch in train_dataloader:
        
    #     with torch.no_grad():
    #         b_input_ids = batch[0].to(device)
    #         b_attention_mask = batch[1].to(device)
    #         b_poses = batch[2].to(device)
    #         b_labels = batch[3].to(device)        

    #         last_layer = model_bert(b_input_ids, b_attention_mask)
    #         b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
    #         gathered_activations = torch.gather(last_layer, 1, b_poses)
    #         logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
    #         loss = loss_fn(logits, b_labels.type_as(logits))
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.cpu().detach().numpy()
    #     label_ids = b_labels.cpu().numpy()

    #     t_labels.append(label_ids)
    #     t_preds.append(logits)
        

    # all_dev_labels = np.concatenate(t_labels, axis=0)
    # all_dev_logits = np.concatenate(t_preds, axis=0)
    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    # print("Train  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
    # print("  Training Validation took: {:}".format(validation_time))





print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## 3.2 Roberta signal en MCL-rev

In [ ]:
train_df = pd.read_csv(path_mcl_en_rev)
train_df.columns = ['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag','id']
train_df = train_df[['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag']]

In [ ]:
dev_df = pd.read_csv(dev_en_rev_path)

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

from transformers import RobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-large')


# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []

list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])

for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)

print('Signal Added .. .')

encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)


# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)


train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)


print('Wordpiece indices ...')

train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)
dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import RobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "roberta-large"
        hidden_states = False
        self.encoder = RobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer



class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x


model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)


optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )


def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)


def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)



path_this = path_save_3_2
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 14
best_val_accuracy = 0
t_best_val_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step == len(train_dataloader)/2:
          print(" Step Half")
          print("")
          print("Running Validation...")

          t0 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model_bert.eval()
          model_log_reg.eval()

          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch in dev_dataloader:

              with torch.no_grad():
                  b_input_ids = batch[0].to(device)
                  b_attention_mask = batch[1].to(device)
                  b_poses = batch[2].to(device)
                  b_labels = batch[3].to(device)        

                  last_layer = model_bert(b_input_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              # print(logits)
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          # print(all_dev_logits)
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
          print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
          actual_val = avg_val_accuracy*20

          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(dev_dataloader)
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)

          if(avg_val_accuracy > best_val_accuracy):
            best_val_accuracy = avg_val_accuracy
            best_val_preds = all_dev_logits
            np.save(os.path.join(path_this, "dev_preds"), best_val_preds)
            torch.save(model_bert.state_dict(), os.path.join(path_this, "roberta_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_this, "log_reg_check_best_dev"))
          
          print("  Validation Loss: {0:.3f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))


          model_bert.train()
          model_log_reg.train()



        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        optimizer.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        last_layer = model_bert(b_input_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:

        with torch.no_grad():
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_poses = batch[2].to(device)
            b_labels = batch[3].to(device)        

            last_layer = model_bert(b_input_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      np.save(os.path.join(path_this, "dev_preds"), best_val_preds)
      torch.save(model_bert.state_dict(), os.path.join(path_this, "roberta_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_this, "log_reg_check_best_dev"))
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

 
    # # Do same on training data
    # # Tracking variables 
    # total_eval_accuracy = 0
    # total_eval_loss = 0
    # nb_eval_steps = 0

    # t_labels = []
    # t_preds = []

    # # Evaluate data for one epoch
    # for batch in train_dataloader:
        
    #     with torch.no_grad():
    #         b_input_ids = batch[0].to(device)
    #         b_attention_mask = batch[1].to(device)
    #         b_poses = batch[2].to(device)
    #         b_labels = batch[3].to(device)        

    #         last_layer = model_bert(b_input_ids, b_attention_mask)
    #         b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
    #         gathered_activations = torch.gather(last_layer, 1, b_poses)
    #         logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
    #         loss = loss_fn(logits, b_labels.type_as(logits))
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.cpu().detach().numpy()
    #     label_ids = b_labels.cpu().numpy()

    #     t_labels.append(label_ids)
    #     t_preds.append(logits)
        

    # all_dev_labels = np.concatenate(t_labels, axis=0)
    # all_dev_logits = np.concatenate(t_preds, axis=0)
    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    # print("Train  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
    # print("  Training Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## 3.3 Roberta signal en Aug en

In [ ]:
train_ausem_df = pd.read_csv(ausemcor_supersence_path).iloc[:,1:]
train_ausem_df.columns = ['lemma', 'pos', 'sent1', 'sent2', 'start1', 'end1', 'start2', 'end2', 'tag', 'position1', 'position2' ]

train_wic_df = pd.read_csv(wic_train_new_format_path)
train_mcl_df = pd.read_csv(path_mcl_en_rev)
train_mcl_df.columns = ['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag','id']
train_mcl_df = train_mcl_df[['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag']]

train_df = pd.concat([train_mcl_df, train_ausem_df, train_wic_df], ignore_index=True)

In [ ]:
dev_df = pd.read_csv(dev_en_rev_path)

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

from transformers import RobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-large')

# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []

list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])

for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)

print('Signal Added ....')


encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)


# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)


train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

print('Wordpiece indices ....')
# labels = torch.from_numpy(train_gold_df['label'].values)
train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)

dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 4

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )



from transformers import RobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "roberta-large"
        hidden_states = False
        self.encoder = RobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer



class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x
model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)


optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 5e-6, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )


def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)

def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)



path_this = path_save_3_3

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 10
best_val_accuracy = 0
t_best_val_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step == len(train_dataloader)/2:
          print(" Step Half")
          print("")
          print("Running Validation...")

          t0 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model_bert.eval()
          model_log_reg.eval()

          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch in dev_dataloader:

              with torch.no_grad():
                  b_input_ids = batch[0].to(device)
                  b_attention_mask = batch[1].to(device)
                  b_poses = batch[2].to(device)
                  b_labels = batch[3].to(device)        

                  last_layer = model_bert(b_input_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              # print(logits)
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          # print(all_dev_logits)
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
          print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
          actual_val = avg_val_accuracy*20

          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(dev_dataloader)
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)

          if(avg_val_accuracy > best_val_accuracy):
            best_val_accuracy = avg_val_accuracy
            best_val_preds = all_dev_logits
            np.save(os.path.join(path_this, "dev_preds"), best_val_preds)
            torch.save(model_bert.state_dict(), os.path.join(path_this, "roberta_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_this, "log_reg_check_best_dev"))
          
          print("  Validation Loss: {0:.3f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))

          

          model_bert.train()
          model_log_reg.train()



        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        optimizer.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        last_layer = model_bert(b_input_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:

        with torch.no_grad():
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_poses = batch[2].to(device)
            b_labels = batch[3].to(device)        

            last_layer = model_bert(b_input_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      np.save(os.path.join(path_this, "dev_preds"), best_val_preds)
      torch.save(model_bert.state_dict(), os.path.join(path_this, "roberta_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_this, "log_reg_check_best_dev"))
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))




    # # Do same on training data
    # # Tracking variables 
    # total_eval_accuracy = 0
    # total_eval_loss = 0
    # nb_eval_steps = 0

    # t_labels = []
    # t_preds = []

    # # Evaluate data for one epoch
    # for batch in train_dataloader:
        
    #     with torch.no_grad():
    #         b_input_ids = batch[0].to(device)
    #         b_attention_mask = batch[1].to(device)
    #         b_poses = batch[2].to(device)
    #         b_labels = batch[3].to(device)        

    #         last_layer = model_bert(b_input_ids, b_attention_mask)
    #         b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
    #         gathered_activations = torch.gather(last_layer, 1, b_poses)
    #         logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
    #         loss = loss_fn(logits, b_labels.type_as(logits))
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.cpu().detach().numpy()
    #     label_ids = b_labels.cpu().numpy()

    #     t_labels.append(label_ids)
    #     t_preds.append(logits)
        

    # all_dev_labels = np.concatenate(t_labels, axis=0)
    # all_dev_logits = np.concatenate(t_preds, axis=0)
    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    # print("Train  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
    # print("  Training Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## 3.4 Bert Large Case MCL-rev

In [ ]:
train_df = pd.read_csv(path_train_mcl)
train_df.columns = ['id', 'lemma', 'pos', 'sent1', 'sent2', 'start1', 'end1', 'start2', 'end2', 'tag', 'position1', 'position2']
train_df = train_df[['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag']]



In [ ]:
dev_df = pd.read_csv(dev_en_rev_path)

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


from transformers import BertTokenizerFast

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizerFast.from_pretrained('bert-large-cased')


# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []

list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])


for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)

print('Signals Added ...')


encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)

print(dev_pos_1[:10])
print(dev_pos_2[:10])

train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

# train_pos_rev_1 = torch.LongTensor(train_pos_rev_1)
# train_pos_rev_2 = torch.LongTensor(train_pos_rev_2)

# train_pos_rev = torch.stack((train_pos_rev_1, train_pos_rev_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

print('Wordpiece indices. ..')

train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)

dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)


from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'], encoded_inputs_train['token_type_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'], encoded_inputs_dev['token_type_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


from transformers import BertModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "bert-large-cased"
        hidden_states = False
        self.encoder = BertModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, token_type_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)

        return last_layer

class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)
# Get all of the model's parameters as a list of tuples.
params = list(model_bert.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )


def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)


import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    print(pred_flat)
    print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)



def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

path_this = path_save_3_4
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))



In [ ]:
import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 10
best_val_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

      # For each batch of training data...
    for step, batch in enumerate(train_dataloader):


        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_poses = batch[3].to(device)
        b_labels = batch[4].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        optimizer.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        last_layer = model_bert(b_input_ids, b_token_type_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
          # scheduler.step()
        
    
    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_poses = batch[3].to(device)
        b_labels = batch[4].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            last_layer = model_bert(b_input_ids, b_token_type_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      np.save(os.path.join(path_this, "dev_preds"), best_val_preds) 
      torch.save(model_bert.state_dict(), os.path.join(path_this, "model_bert_dev_best"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_this, "model_log_reg_dev_best"))
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
    
    # Do same on test data
    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in train_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_poses = batch[3].to(device)
        b_labels = batch[4].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            last_layer = model_bert(b_input_ids, b_token_type_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    print("Train Accuracy: {0:.4f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    # Do same on training data
    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # # Evaluate data for one epoch
    # for batch in train_dataloader:
        
    #     # Unpack this training batch from our dataloader. 
    #     #
    #     # As we unpack the batch, we'll also copy each tensor to the GPU using 
    #     # the `to` method.

    #     b_input_ids = batch[0].to(device)
    #     b_token_type_ids = batch[1].to(device)
    #     b_attention_mask = batch[2].to(device)
    #     b_poses = batch[3].to(device)
    #     b_labels = batch[4].to(device)
        
    #     # Tell pytorch not to bother with constructing the compute graph during
    #     # the forward pass, since this is only needed for backprop (training).
    #     with torch.no_grad():
    #         last_layer = model_bert(b_input_ids, b_token_type_ids, b_attention_mask)
    #         b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
    #         gathered_activations = torch.gather(last_layer, 1, b_poses)
    #         logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
    #         loss = loss_fn(logits, b_labels.type_as(logits))
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.cpu().detach().numpy()
    #     label_ids = b_labels.cpu().numpy()

    #     t_labels.append(label_ids)
    #     t_preds.append(logits)
        

    # all_dev_labels = np.concatenate(t_labels, axis=0)
    # all_dev_logits = np.concatenate(t_preds, axis=0)
    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    # print("Train  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
    # print("  Training Validation took: {:}".format(validation_time))



print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

# **4. Training Models - Multilingual**

## 4.1  mBERT 8k en non rev + 750 per lang rev


In [ ]:
train_mcl_df = pd.read_csv(path_train_mcl).iloc[:,1:]
train_mcl_df.columns = ['lemma', 'pos', 'sent1', 'sent2', 'start1', 'end1', 'start2', 'end2', 'tag', 'position1', 'position2']
mix_df = pd.read_csv(path_mix6k_train_rev).iloc[:,3:]
mix_df.rename(columns={"index1":"position1", "index2":"position2"}, inplace=True)
train_df = pd.concat([train_mcl_df, mix_df], ignore_index=True)

In [ ]:
dev_df_en = pd.read_csv(devfinal_en_path)
dev_df_en.rename(columns={"sentence1":"sent1", "sentence2":"sent2"}, inplace=True)
dev_df_mix = pd.read_csv(devmix_200_rev_path).iloc[:,1:].rename(columns={"sentence1":"sent1", "sentence2":"sent2"})
dev_df = pd.concat([dev_df_en, dev_df_mix], ignore_index=True)

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

from transformers import BertTokenizerFast

# Load the BERT tokenizer.
print('Loading bert tokenizer...')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')


# sentences is a list 0f str

# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []



list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])


for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)


print("Signal Added ...")

encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)


# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)


train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

print("wordpiece indices done ...")

train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)
dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'], encoded_inputs_train['token_type_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'], encoded_inputs_dev['token_type_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import BertModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "bert-base-multilingual-cased"
        hidden_states = False
        self.encoder = BertModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, token_type_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)

        return last_layer



class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(1536, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)

optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )

def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)

def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat.shape)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


path_mix = path_mix_4_2


import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
# import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 10
best_val_accuracy = 0
t_best_val_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # if epoch_i ==0:
        #   break

        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step == int(len(train_dataloader)/3) or step == int(2*len(train_dataloader)/3):
          print(" Step 1/3rds")
          print("")
          print("Running Validation...")

          t0 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model_bert.eval()
          model_log_reg.eval()

          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch in dev_dataloader:

              with torch.no_grad():
                  b_input_ids = batch[0].to(device)
                  b_token_type_ids = batch[1].to(device)
                  b_attention_mask = batch[2].to(device)
                  b_poses = batch[3].to(device)
                  b_labels = batch[4].to(device)        

                  last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              # print(logits)
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          # print(all_dev_logits)
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
          print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
          en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])
          
          
          ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
          zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
          fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
          ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
          print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
          print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
          print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
          print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
          print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))



          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(dev_dataloader)
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)

          if(avg_val_accuracy > best_val_accuracy):
            flag = True
            best_val_accuracy = avg_val_accuracy
            best_val_preds = all_dev_logits
            np.save(os.path.join(path_mix, "dev_preds"), best_val_preds)
            torch.save(model_bert.state_dict(), os.path.join(path_mix, "mbert_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
          
          print("  Validation Loss: {0:.3f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))

          model_bert.train()
          model_log_reg.train()

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_poses = batch[3].to(device)
        b_labels = batch[4].to(device)        

        optimizer.zero_grad()        

        last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:

        with torch.no_grad():
            b_input_ids = batch[0].to(device)
            b_token_type_ids = batch[1].to(device)
            b_attention_mask = batch[2].to(device)
            b_poses = batch[3].to(device)
            b_labels = batch[4].to(device)        

            last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])


    ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
    zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
    fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
    ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
    print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
    print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
    print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
    print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
    print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_mix, "mbert_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
      np.save(os.path.join(path_mix, "best_dev_preds"), best_val_preds)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))



print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## 4.2 mBERT En non rev + farsi + danish + croation + chinese + mcl750 rev 18k points


In [ ]:
train_mcl_df = pd.read_csv(path_train_mcl).iloc[:,1:]
train_mcl_df.columns = ['lemma', 'pos', 'sent1', 'sent2', 'start1', 'end1', 'start2', 'end2', 'tag', 'position1', 'position2']

mix_df = pd.read_csv(path_mix6k_train_rev).iloc[:,3:]
mix_df.rename(columns={"index1":"position1", "index2":"position2"}, inplace=True)

farsi_df = pd.read_csv(farsi_df_path).iloc[:,:]
farsi_df.rename(columns={"index1":"position1", "index2":"position2"}, inplace=True)
xl_zh_df = pd.read_csv(xl_zh_df_path)
xl_nl_df = pd.read_csv(xl_nl_df_path)
xl_hr_df = pd.read_csv(xl_hr_df_path)
xl_da_df = pd.read_csv(xl_da_df_path)
train_df = pd.concat([train_mcl_df, mix_df, xl_da_df, xl_hr_df, xl_nl_df, xl_zh_df, farsi_df], ignore_index=True)

In [ ]:
dev_df_en = pd.read_csv(devfinal_en_path)
dev_df_en.rename(columns={"sentence1":"sent1", "sentence2":"sent2"}, inplace=True)
dev_df_mix = pd.read_csv(devmix_200_rev_path).iloc[:,1:].rename(columns={"sentence1":"sent1", "sentence2":"sent2"})
dev_df = pd.concat([dev_df_en, dev_df_mix], ignore_index=True)

In [ ]:

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



from transformers import BertTokenizerFast

# Load the BERT tokenizer.
print('Loading bert tokenizer...')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')



# sentences is a list 0f str

# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []



list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])


for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)

print("Signal added ....")


encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)


# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)


train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

print('wordpiece indices ...')

train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)

dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)


from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'], encoded_inputs_train['token_type_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'], encoded_inputs_dev['token_type_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import BertModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "bert-base-multilingual-cased"
        hidden_states = False
        self.encoder = BertModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, token_type_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)

        return last_layer


class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(1536, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x


model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)

optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )


def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)

def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat.shape)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)



path_mix = path_mix_4_3

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
# import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 15
best_val_accuracy = 0
t_best_val_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # if epoch_i ==0:
        #   break

        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step == int(len(train_dataloader)/3) or step == int(2*len(train_dataloader)/3):
          print(" Step 1/3rds")
          print("")
          print("Running Validation...")

          t0 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model_bert.eval()
          model_log_reg.eval()

          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch in dev_dataloader:

              with torch.no_grad():
                  b_input_ids = batch[0].to(device)
                  b_token_type_ids = batch[1].to(device)
                  b_attention_mask = batch[2].to(device)
                  b_poses = batch[3].to(device)
                  b_labels = batch[4].to(device)        

                  last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              # print(logits)
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          # print(all_dev_logits)
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
          print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
          en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])
          
          
          ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
          zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
          fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
          ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
          print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
          print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
          print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
          print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
          print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))



          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(dev_dataloader)
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)

          if(avg_val_accuracy > best_val_accuracy):
            flag = True
            best_val_accuracy = avg_val_accuracy
            best_val_preds = all_dev_logits
            np.save(os.path.join(path_mix, "dev_preds"), best_val_preds)
            torch.save(model_bert.state_dict(), os.path.join(path_mix, "mbert_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
          
          print("  Validation Loss: {0:.3f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))

          model_bert.train()
          model_log_reg.train()

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_poses = batch[3].to(device)
        b_labels = batch[4].to(device)        

        optimizer.zero_grad()        

        last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:

        with torch.no_grad():
            b_input_ids = batch[0].to(device)
            b_token_type_ids = batch[1].to(device)
            b_attention_mask = batch[2].to(device)
            b_poses = batch[3].to(device)
            b_labels = batch[4].to(device)        

            last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])


    ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
    zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
    fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
    ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
    print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
    print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
    print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
    print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
    print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_mix, "mbert_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
      np.save(os.path.join(path_mix, "best_dev_preds"), best_val_preds)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))


    
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## 4.3 xlmr 8k en rev + 750 per langrev

In [ ]:
train_mcl_df = pd.read_csv(path_training)
train_mcl_df.columns = ['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag','id']
train_mcl_df = train_mcl_df[['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag']]

mix_df = pd.read_csv(path_mix6k_train_rev).iloc[:,3:]
mix_df.rename(columns={"index1":"position1", "index2":"position2"}, inplace=True)

train_df = pd.concat([train_mcl_df, mix_df], ignore_index=True)

In [ ]:
dev_df_en = pd.read_csv(devfinal_en_path)
dev_df_en.rename(columns={"sentence1":"sent1", "sentence2":"sent2"}, inplace=True)

dev_df_mix = pd.read_csv(devmix_200_rev_path).iloc[:,1:].rename(columns={"sentence1":"sent1", "sentence2":"sent2"})
dev_df = pd.concat([dev_df_en, dev_df_mix], ignore_index=True)


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


from transformers import XLMRobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-large')



# sentences is a list 0f str

# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []



list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])


for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)


print('Signal Added ...')

encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)


# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)


train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)


print('wordpiece indexing done....')


train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)

dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)


from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)



from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


from transformers import XLMRobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "xlm-roberta-large"
        hidden_states = False
        self.encoder = XLMRobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer




class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x


model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)

optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )



def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)


def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat.shape)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)



path_mix = path_mix_4_4


import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 10
best_val_accuracy = 0
t_best_val_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # if epoch_i ==0:
        #   break

        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step == int(len(train_dataloader)/3) or step == int(2*len(train_dataloader)/3):
          print(" Step 1/3rds")
          print("")
          print("Running Validation...")

          t0 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model_bert.eval()
          model_log_reg.eval()

          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch in dev_dataloader:

              with torch.no_grad():
                  b_input_ids = batch[0].to(device)
                  b_attention_mask = batch[1].to(device)
                  b_poses = batch[2].to(device)
                  b_labels = batch[3].to(device)        

                  last_layer = model_bert(b_input_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              # print(logits)
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          # print(all_dev_logits)
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
          print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
          en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])
          
          
          ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
          zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
          fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
          ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
          print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
          print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
          print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
          print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
          print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))



          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(dev_dataloader)
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)

          if(avg_val_accuracy > best_val_accuracy):
            flag = True
            best_val_accuracy = avg_val_accuracy
            best_val_preds = all_dev_logits
            np.save(os.path.join(path_mix, "dev_preds"), best_val_preds)
            torch.save(model_bert.state_dict(), os.path.join(path_mix, "roberta_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
          
          print("  Validation Loss: {0:.3f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))

          model_bert.train()
          model_log_reg.train()

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        optimizer.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        last_layer = model_bert(b_input_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:

        with torch.no_grad():
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_poses = batch[2].to(device)
            b_labels = batch[3].to(device)        

            last_layer = model_bert(b_input_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])


    ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
    zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
    fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
    ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
    print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
    print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
    print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
    print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
    print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      flag = True
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_mix, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
      np.save(os.path.join(path_mix, "best_dev_preds"), best_val_preds)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))


    

    # # Do same on training data
    # # Tracking variables 
    # total_eval_accuracy = 0
    # total_eval_loss = 0
    # nb_eval_steps = 0

    # t_labels = []
    # t_preds = []

    # # Evaluate data for one epoch
    # for batch in train_dataloader:
        
    #     with torch.no_grad():
    #         b_input_ids = batch[0].to(device)
    #         b_attention_mask = batch[1].to(device)
    #         b_poses = batch[2].to(device)
    #         b_labels = batch[3].to(device)        

    #         last_layer = model_bert(b_input_ids, b_attention_mask)
    #         b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
    #         gathered_activations = torch.gather(last_layer, 1, b_poses)
    #         logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
    #         loss = loss_fn(logits, b_labels.type_as(logits))
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.cpu().detach().numpy()
    #     label_ids = b_labels.cpu().numpy()

    #     t_labels.append(label_ids)
    #     t_preds.append(logits)
        

    # all_dev_labels = np.concatenate(t_labels, axis=0)
    # all_dev_logits = np.concatenate(t_preds, axis=0)
    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    # print("Train  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
    # print("  Training Validation took: {:}".format(validation_time))





print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## 4.4 xlmr En rev + farsi + danish + croation + chinese + mcl750 rev

In [ ]:
train_mcl_df = pd.read_csv(path_mcl_en_rev)
train_mcl_df.columns = ['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag','id']
train_mcl_df = train_mcl_df[['lemma', 'pos', 'position1', 'start1', 'end1', 'position2', 'start2', 'end2', 'sent1', 'sent2', 'tag']]

mix_df = pd.read_csv(path_mix6k_train_rev).iloc[:,3:]
mix_df.rename(columns={"index1":"position1", "index2":"position2"}, inplace=True)

farsi_df = pd.read_csv(farsi_df_path).iloc[:,:]
farsi_df.rename(columns={"index1":"position1", "index2":"position2"}, inplace=True)
xl_zh_df = pd.read_csv(xl_zh_df_path)
xl_nl_df = pd.read_csv(xl_nl_df_path)
xl_hr_df = pd.read_csv(xl_hr_df_path)
xl_da_df = pd.read_csv(xl_da_df_path)

train_df = pd.concat([train_mcl_df, mix_df, xl_da_df, xl_hr_df, xl_nl_df, xl_zh_df, farsi_df], ignore_index=True)


In [ ]:
dev_df_en = pd.read_csv(devfinal_en_path)
dev_df_en.rename(columns={"sentence1":"sent1", "sentence2":"sent2"}, inplace=True)

dev_df_mix = pd.read_csv(devmix_200_rev_path).iloc[:,1:].rename(columns={"sentence1":"sent1", "sentence2":"sent2"})
dev_df = pd.concat([dev_df_en, dev_df_mix], ignore_index=True)


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


from transformers import XLMRobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-large')


# sentences is a list 0f str

# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []



list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])


for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)



print('Signal Added ....')


encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)


train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)


print('Wordpiece indices ...')

train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)

dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


from transformers import XLMRobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "xlm-roberta-large"
        hidden_states = False
        self.encoder = XLMRobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer


class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x



model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)


optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )


def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)


def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat.shape)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


path_mix = path_mix_4_5

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))




In [ ]:
# import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 10
best_val_accuracy = 0
t_best_val_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # if epoch_i ==0:
        #   break

        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step == int(len(train_dataloader)/3) or step == int(2*len(train_dataloader)/3):
          print(" Step 1/3rds")
          print("")
          print("Running Validation...")

          t0 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model_bert.eval()
          model_log_reg.eval()

          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch in dev_dataloader:

              with torch.no_grad():
                  b_input_ids = batch[0].to(device)
                  b_attention_mask = batch[1].to(device)
                  b_poses = batch[2].to(device)
                  b_labels = batch[3].to(device)        

                  last_layer = model_bert(b_input_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              # print(logits)
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          # print(all_dev_logits)
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
          print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
          en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])
          
          
          ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
          zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
          fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
          ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
          print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
          print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
          print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
          print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
          print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))



          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(dev_dataloader)
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)

          if(avg_val_accuracy > best_val_accuracy):
            flag = True
            best_val_accuracy = avg_val_accuracy
            best_val_preds = all_dev_logits
            np.save(os.path.join(path_mix, "dev_preds"), best_val_preds)
            torch.save(model_bert.state_dict(), os.path.join(path_mix, "roberta_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
          
          print("  Validation Loss: {0:.3f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))

          model_bert.train()
          model_log_reg.train()

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        optimizer.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        last_layer = model_bert(b_input_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:

        with torch.no_grad():
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_poses = batch[2].to(device)
            b_labels = batch[3].to(device)        

            last_layer = model_bert(b_input_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])


    ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
    zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
    fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
    ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
    print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
    print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
    print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
    print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
    print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      flag = True
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_mix, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
      np.save(os.path.join(path_mix, "best_dev_preds"), best_val_preds)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))


    

    # # Do same on training data
    # # Tracking variables 
    # total_eval_accuracy = 0
    # total_eval_loss = 0
    # nb_eval_steps = 0

    # t_labels = []
    # t_preds = []

    # # Evaluate data for one epoch
    # for batch in train_dataloader:
        
    #     with torch.no_grad():
    #         b_input_ids = batch[0].to(device)
    #         b_attention_mask = batch[1].to(device)
    #         b_poses = batch[2].to(device)
    #         b_labels = batch[3].to(device)        

    #         last_layer = model_bert(b_input_ids, b_attention_mask)
    #         b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
    #         gathered_activations = torch.gather(last_layer, 1, b_poses)
    #         logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
    #         loss = loss_fn(logits, b_labels.type_as(logits))
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.cpu().detach().numpy()
    #     label_ids = b_labels.cpu().numpy()

    #     t_labels.append(label_ids)
    #     t_preds.append(logits)
        

    # all_dev_labels = np.concatenate(t_labels, axis=0)
    # all_dev_logits = np.concatenate(t_preds, axis=0)
    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    # print("Train  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
    # print("  Training Validation took: {:}".format(validation_time))





print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## 4.5 xlmr 8k en non rev + 750 per lang rev


In [ ]:
train_mcl_df = pd.read_csv(path_train_mcl).iloc[:,1:]
train_mcl_df.columns = ['lemma', 'pos', 'sent1', 'sent2', 'start1', 'end1', 'start2', 'end2', 'tag', 'position1', 'position2']

mix_df = pd.read_csv(path_mix6k_train_rev).iloc[:,3:]
mix_df.rename(columns={"index1":"position1", "index2":"position2"}, inplace=True)

train_df = pd.concat([train_mcl_df, mix_df], ignore_index=True)

In [ ]:
dev_df_en = pd.read_csv(devfinal_en_path)
dev_df_en.rename(columns={"sentence1":"sent1", "sentence2":"sent2"}, inplace=True)

dev_df_mix = pd.read_csv(devmix_200_rev_path).iloc[:,1:].rename(columns={"sentence1":"sent1", "sentence2":"sent2"})

dev_df = pd.concat([dev_df_en, dev_df_mix], ignore_index=True)

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



from transformers import XLMRobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-large')


# sentences is a list 0f str

# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []



list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])


for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)


print('Signal Added ....')

encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)

# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)


train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

print('Wordpiece Indices ...')


train_gold_df_tmp = train_df[['tag']].replace({'F' : 0, 'T' : 1})
train_labels = torch.from_numpy(train_gold_df_tmp.values)

dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos, train_labels)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import XLMRobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "xlm-roberta-large"
        hidden_states = False
        self.encoder = XLMRobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer


class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x


model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)
optimizer = AdamW(list(model_bert.parameters()) + list(model_log_reg.parameters()),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 1e-5
                  eps = 1e-8 ,
                  # weight_decay = 1 # args.adam_epsilon  - default is 1e-8.
                  )

def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)


def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat.shape)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

path_mix = path_mix_4_6



import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
# import random
# import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
epochs = 10
best_val_accuracy = 0
t_best_val_accuracy = 0
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()
    model_log_reg.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # if epoch_i ==0:
        #   break

        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        if step == int(len(train_dataloader)/3) or step == int(2*len(train_dataloader)/3):
          print(" Step 1/3rds")
          print("")
          print("Running Validation...")

          t0 = time.time()

          # Put the model in evaluation mode--the dropout layers behave differently
          # during evaluation.
          model_bert.eval()
          model_log_reg.eval()

          # Tracking variables 
          total_eval_accuracy = 0
          total_eval_loss = 0
          nb_eval_steps = 0

          t_labels = []
          t_preds = []

          # Evaluate data for one epoch
          for batch in dev_dataloader:

              with torch.no_grad():
                  b_input_ids = batch[0].to(device)
                  b_attention_mask = batch[1].to(device)
                  b_poses = batch[2].to(device)
                  b_labels = batch[3].to(device)        

                  last_layer = model_bert(b_input_ids, b_attention_mask)
                  b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
                  gathered_activations = torch.gather(last_layer, 1, b_poses)
                  logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
                  loss = loss_fn(logits, b_labels.type_as(logits))
                  
              # Accumulate the validation loss.
              total_eval_loss += loss.item()

              # Move logits and labels to CPU
              logits = logits.cpu().detach().numpy()
              # print(logits)
              label_ids = b_labels.cpu().numpy()

              t_labels.append(label_ids)
              t_preds.append(logits)
              

          all_dev_labels = np.concatenate(t_labels, axis=0)
          all_dev_logits = np.concatenate(t_preds, axis=0)
          # Report the final accuracy for this validation run.
          # print(all_dev_logits)
          avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
          print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
          en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])
          
          
          ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
          zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
          fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
          ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
          print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
          print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
          print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
          print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
          print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))



          # Calculate the average loss over all of the batches.
          avg_val_loss = total_eval_loss / len(dev_dataloader)
          
          # Measure how long the validation run took.
          validation_time = format_time(time.time() - t0)

          if(avg_val_accuracy > best_val_accuracy):
            flag = True
            best_val_accuracy = avg_val_accuracy
            best_val_preds = all_dev_logits
            np.save(os.path.join(path_mix, "dev_preds"), best_val_preds)
            torch.save(model_bert.state_dict(), os.path.join(path_mix, "roberta_check_best_dev"))
            torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
          
          print("  Validation Loss: {0:.3f}".format(avg_val_loss))
          print("  Validation took: {:}".format(validation_time))

          model_bert.train()
          model_log_reg.train()

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.

        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        optimizer.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        last_layer = model_bert(b_input_ids, b_attention_mask)

        # print(b_poses[:,0].size())
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        # print(b_poses.size())
        gathered_activations = torch.gather(last_layer, 1, b_poses)

        # concatted = torch.cat((last_layer[b_poses[:,0].view(16,1),:] , last_layer[:,b_poses[:,1],:]), dim = 1)

        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        # print(type(b_labels))
        loss = loss_fn(logits, b_labels.type_as(logits))
        # print("loss = "+str(loss))
        # print(logits.cpu().detach().numpy())

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        # print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        # TODO logsistic REG????/
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # # Update the learning rate.
        # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()
    model_log_reg.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    t_labels = []
    t_preds = []

    # Evaluate data for one epoch
    for batch in dev_dataloader:

        with torch.no_grad():
            b_input_ids = batch[0].to(device)
            b_attention_mask = batch[1].to(device)
            b_poses = batch[2].to(device)
            b_labels = batch[3].to(device)        

            last_layer = model_bert(b_input_ids, b_attention_mask)
            b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
            gathered_activations = torch.gather(last_layer, 1, b_poses)
            logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
            loss = loss_fn(logits, b_labels.type_as(logits))
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.cpu().detach().numpy()
        # print(logits)
        label_ids = b_labels.cpu().numpy()

        t_labels.append(label_ids)
        t_preds.append(logits)
        

    all_dev_labels = np.concatenate(t_labels, axis=0)
    all_dev_logits = np.concatenate(t_preds, axis=0)
    # Report the final accuracy for this validation run.
    # print(all_dev_logits)
    avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits[200:], all_dev_labels[200:])
    print("Validation Accuracy: {0:.4f}".format(avg_val_accuracy))
    en_dev_acc = flat_accuracy_single_logit(all_dev_logits[:200], all_dev_labels[:200])


    ar_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,:50].ravel(), all_dev_labels[200:].reshape(2,200)[:,:50].ravel())
    zh_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,50:100].ravel(), all_dev_labels[200:].reshape(2,200)[:,50:100].ravel())
    fr_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,100:150].ravel(), all_dev_labels[200:].reshape(2,200)[:,100:150].ravel())
    ru_dev_acc = flat_accuracy_single_logit(all_dev_logits[200:].reshape(2,200)[:,150:200].ravel(), all_dev_labels[200:].reshape(2,200)[:,150:200].ravel())
    print("EN dev Accuracy: {0:.4f}".format(en_dev_acc), end = ",")
    print("RU dev Accuracy: {0:.4f} ".format(ru_dev_acc), end = ",")
    print("ZH dev Accuracy: {0:.4f} ".format(zh_dev_acc), end = ",")
    print("AR dev Accuracy: {0:.4f} ".format(ar_dev_acc), end = ",")
    print("FR dev Accuracy: {0:.4f} ".format(fr_dev_acc))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(dev_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    if(avg_val_accuracy > best_val_accuracy):
      flag = True
      best_val_accuracy = avg_val_accuracy
      best_val_preds = all_dev_logits
      torch.save(model_bert.state_dict(), os.path.join(path_mix, "xlmr_check_best_dev"))
      torch.save(model_log_reg.state_dict(), os.path.join(path_mix, "log_reg_check_best_dev"))
      np.save(os.path.join(path_mix, "best_dev_preds"), best_val_preds)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))


    

    # # Do same on training data
    # # Tracking variables 
    # total_eval_accuracy = 0
    # total_eval_loss = 0
    # nb_eval_steps = 0

    # t_labels = []
    # t_preds = []

    # # Evaluate data for one epoch
    # for batch in train_dataloader:
        
    #     with torch.no_grad():
    #         b_input_ids = batch[0].to(device)
    #         b_attention_mask = batch[1].to(device)
    #         b_poses = batch[2].to(device)
    #         b_labels = batch[3].to(device)        

    #         last_layer = model_bert(b_input_ids, b_attention_mask)
    #         b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
    #         gathered_activations = torch.gather(last_layer, 1, b_poses)
    #         logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
    #         loss = loss_fn(logits, b_labels.type_as(logits))
            
    #     # Accumulate the validation loss.
    #     total_eval_loss += loss.item()

    #     # Move logits and labels to CPU
    #     logits = logits.cpu().detach().numpy()
    #     label_ids = b_labels.cpu().numpy()

    #     t_labels.append(label_ids)
    #     t_preds.append(logits)
        

    # all_dev_labels = np.concatenate(t_labels, axis=0)
    # all_dev_logits = np.concatenate(t_preds, axis=0)
    # # Report the final accuracy for this validation run.
    # avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
    # print("Train  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # # Calculate the average loss over all of the batches.
    # avg_val_loss = total_eval_loss / len(train_dataloader)
    
    # # Measure how long the validation run took.
    # validation_time = format_time(time.time() - t0)
    
    # print("  Train Avg Loss: {0:.3f}".format(avg_val_loss))
    # print("  Training Validation took: {:}".format(validation_time))





print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

# 5. **Predicting labels for given test set**

## 5.1 MultiLingual Inference Codes

In [ ]:
# folder paths of saved models
path_mbert_1_1 = "/content/drive/MyDrive/datasets/sub/models/Mix/mbert_mix_no_en_no_revmcl_1/"
path_mbert_1_2 = "/content/drive/MyDrive/datasets/sub/models/Mix/mbert_mix_no_en_no_revmcl_xlwic_1/"

path_xlmr_1_1 = "/content/drive/MyDrive/datasets/sub/models/Mix/xlmr_mix_no_en_rev/"
path_xlmr_1_2 = "/content/drive/MyDrive/datasets/sub/models/Mix/xlmr_mix_no_en_wic_xlwic/"
path_xlmr_1_3 = "/content/drive/MyDrive/datasets/final/models/Mix/xlmr_mix_no_en_5/"

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/datasets/final/datasets/Non-English/nonenglishtest.csv")
test_df.rename(columns={"sentence1":"sent1", "sentence2":"sent2"}, inplace=True)
print(test_df['id'])
test_df.head()

In [ ]:
dev_df = pd.read_csv("/content/drive/MyDrive/datasets/final/datasets/Mix/dev_mix_200_rev.csv")
dev_df.rename(columns={"sentence1":"sent1", "sentence2":"sent2"}, inplace=True)
tags = dev_df['tag'].to_numpy()
print(dev_df)

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

In [ ]:
# Adding a signal without spaces
test_sentences_1 =  []
test_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []

list_test_sentences_1 = list(test_df['sent1'])
list_test_sentences_2 = list(test_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])

for i in range(len(list_test_sentences_1)):
  sentence_1 = list_test_sentences_1[i]
  # print(sentence_1)
  s1 = int(test_df['start1'][i])
  e1 = int(test_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  test_sentences_1.append(sentence_1)

  sentence_2 = list_test_sentences_2[i]
  # print(sentence_2)
  s2 = int(test_df['start2'][i])
  e2 = int(test_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  test_sentences_2.append(sentence_2)
  # print((test_sentences_1[i], test_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)


print((test_sentences_2[:10]))
print((test_sentences_1[:10]))
print(dev_sentences_1[:10])
print(dev_sentences_2[:10])


dev_gold_df_tmp = dev_df[['tag']].replace({'F' : 0, 'T' : 1})
dev_labels = torch.from_numpy(dev_gold_df_tmp.values)

['Утверждение о том, что другие тюрьмы "находятся" в аналогичном состоянии, можно считать достаточно убедительным.', 'Поэтому мы поддержали тот "путь", который был предложен Председателем, и полностью приняли видоизмененную повестку дня 1996 года в качестве "наименьшего общего знаменателя".', 'Развитие представляет собой многоаспектную задачу по "повышению" качества жизни всех людей.', 'Совершенно очевидно, что следствием этого может стать ухудшение "качества" услуг, оказываемых БАПОР, а в конечном счете - ухудшение условий жизни беженцев.', 'В ходе подготовки к специальной сессии была также "выпущена" в эфир специальная радиопрограмма в четырех частях об Организации Объединенных Наций на других языках.', 'Как "показано" в приложении III, балансовая стоимость такого проданного имущества составляла 791 400 долл. США, а стоимость реализации, соответственно, 632 900 долл. США.', 'По просьбе Группы Организации Объединенных Наций по оказанию помощи в проведении выборов в Южной Африке и Моза

In [ ]:
def loss_fn(output, targets):
  return nn.BCEWithLogitsLoss(reduction='mean')(output, targets)

In [ ]:
def flat_accuracy_single_logit(preds, labels):
    pred_flat = (preds>0).flatten()
    labels_flat = labels.flatten()
    # print(pred_flat)
    # print(labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

**mBERT Preprocessing**

In [ ]:
from transformers import BertTokenizerFast

# Load the BERT tokenizer.
print('Loading bert tokenizer...')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

encoded_inputs_test = tokenizer(test_sentences_1, test_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)


# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
test_pos_1 = []
test_pos_2 = []

for j in range(len(test_sentences_1)):
  offsets = encoded_inputs_test['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(test_df['start1'][j])
  s2 = int(test_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  test_pos_1.append(pos1)
  test_pos_2.append(pos2)
# print(train_pos_1)
# print(train_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)

test_pos_1 = torch.LongTensor(test_pos_1)
test_pos_2 = torch.LongTensor(test_pos_2)

test_pos = torch.stack((test_pos_1, test_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

from torch.utils.data import TensorDataset

test_dataset = TensorDataset(encoded_inputs_test['input_ids'], encoded_inputs_test['token_type_ids'],
                              encoded_inputs_test['attention_mask'], test_pos)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'], encoded_inputs_dev['token_type_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)

batch_size = 16

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
test_dataloader = DataLoader(
            test_dataset,  # The training samples.
            sampler = SequentialSampler(test_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import BertModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "bert-base-multilingual-cased"
        hidden_states = False
        self.encoder = BertModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, token_type_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)

        return last_layer

class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(1536, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)



**mBERT 1.1**

In [ ]:
model_bert.load_state_dict(torch.load(os.path.join(path_mbert_1_1, 'mbert_check_best_dev')))
model_log_reg.load_state_dict(torch.load(os.path.join(path_mbert_1_1, 'log_reg_check_best_dev')))

model_bert.eval()
model_log_reg.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []

for batch in test_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_poses = batch[3].to(device)

        last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()

    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)
mbert_en_no_rev_test = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []
for batch in dev_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_poses = batch[3].to(device)
        b_labels = batch[4].to(device)        

        last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        loss = loss_fn(logits, b_labels.type_as(logits))
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    # print(logits)
    label_ids = b_labels.cpu().numpy()

    t_labels.append(label_ids)
    t_preds.append(logits)
    

all_dev_labels = np.concatenate(t_labels, axis=0)
all_dev_logits = np.concatenate(t_preds, axis=0)
# Report the final accuracy for this validation run.
# print(all_dev_logits)
avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
          
print("Dev Accuracy: {0:.4f}".format(avg_val_accuracy))

mbert_en_no_rev_dev = 1 / (1 + np.exp(-all_dev_logits))

**mBERT 1.2**

In [ ]:
model_bert.load_state_dict(torch.load(os.path.join(path_mbert_1_2, 'mbert_check_best_dev')))
model_log_reg.load_state_dict(torch.load(os.path.join(path_mbert_1_2, 'log_reg_check_best_dev')))

model_bert.eval()
model_log_reg.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []

for batch in test_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_poses = batch[3].to(device)

        last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()

    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)
mbert_xlwic_test = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []
for batch in dev_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_token_type_ids = batch[1].to(device)
        b_attention_mask = batch[2].to(device)
        b_poses = batch[3].to(device)
        b_labels = batch[4].to(device)        

        last_layer = model_bert(b_input_ids,b_token_type_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 768)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        loss = loss_fn(logits, b_labels.type_as(logits))
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    # print(logits)
    label_ids = b_labels.cpu().numpy()

    t_labels.append(label_ids)
    t_preds.append(logits)
    

all_dev_labels = np.concatenate(t_labels, axis=0)
all_dev_logits = np.concatenate(t_preds, axis=0)
# Report the final accuracy for this validation run.
# print(all_dev_logits)
avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
          
print("Dev Accuracy: {0:.4f}".format(avg_val_accuracy))

mbert_xlwic_dev = 1 / (1 + np.exp(-all_dev_logits))

**XLMR 1.1 with XLMR Preprocessing**

In [ ]:
#xlmr_mcl
from transformers import XLMRobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-large')

encoded_inputs_test = tokenizer(test_sentences_1, test_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_test['input_ids'][0]))
print(type(encoded_inputs_test['input_ids']))


encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
test_pos_1 = []
test_pos_2 = []

for j in range(len(test_sentences_1)):
  offsets = encoded_inputs_test['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(test_df['start1'][j])
  s2 = int(test_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  test_pos_1.append(pos1)
  test_pos_2.append(pos2)
# print(train_pos_1)
# print(test_pos_2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)

test_pos_1 = torch.LongTensor(test_pos_1)
test_pos_2 = torch.LongTensor(test_pos_2)

test_pos = torch.stack((test_pos_1, test_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

from torch.utils.data import TensorDataset

# Combine the testing inputs into a TensorDataset.
test_dataset = TensorDataset(encoded_inputs_test['input_ids'],
                              encoded_inputs_test['attention_mask'], test_pos)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos, dev_labels)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for testing, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take testing samples in random order. 
test_dataloader = DataLoader(
            test_dataset,  # The training samples.
            sampler = SequentialSampler(test_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import XLMRobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "xlm-roberta-large"
        hidden_states = False
        self.encoder = XLMRobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer

class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)



model_bert.load_state_dict(torch.load(os.path.join(path_xlmr_1_1,'roberta_check_best_dev')))
model_log_reg.load_state_dict(torch.load(os.path.join(path_xlmr_1_1, 'log_reg_check_best_dev')))

model_bert.eval()
model_log_reg.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []

for batch in test_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    # print(logits)

    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

xlmr_87_test = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []
for batch in dev_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)        

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        loss = loss_fn(logits, b_labels.type_as(logits))
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    # print(logits)
    label_ids = b_labels.cpu().numpy()

    t_labels.append(label_ids)
    t_preds.append(logits)
    

all_dev_labels = np.concatenate(t_labels, axis=0)
all_dev_logits = np.concatenate(t_preds, axis=0)
# Report the final accuracy for this validation run.
# print(all_dev_logits)
avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
          
print("Dev Accuracy: {0:.4f}".format(avg_val_accuracy))

xlmr_87_dev = 1 / (1 + np.exp(-all_dev_logits))

**XLMR 1.2**

In [ ]:
model_bert.load_state_dict(torch.load(os.path.join(path_xlmr_1_2,'roberta_check_best_dev')))
model_log_reg.load_state_dict(torch.load(os.path.join(path_xlmr_1_2, 'log_reg_check_best_dev')))

model_bert.eval()
model_log_reg.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []

for batch in test_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    # print(logits)

    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

xlmr_xlwic_test = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []
for batch in dev_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)        

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        loss = loss_fn(logits, b_labels.type_as(logits))
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    # print(logits)
    label_ids = b_labels.cpu().numpy()

    t_labels.append(label_ids)
    t_preds.append(logits)
    

all_dev_labels = np.concatenate(t_labels, axis=0)
all_dev_logits = np.concatenate(t_preds, axis=0)
# Report the final accuracy for this validation run.
# print(all_dev_logits)
avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
          
print("Dev Accuracy: {0:.4f}".format(avg_val_accuracy))

xlmr_xlwic_dev = 1 / (1 + np.exp(-all_dev_logits))

**XLMR 1.3**

In [ ]:
model_bert.load_state_dict(torch.load(os.path.join(path_xlmr_1_3,'roberta_check_best_dev')))
model_log_reg.load_state_dict(torch.load(os.path.join(path_xlmr_1_3, 'log_reg_check_best_dev')))

model_bert.eval()
model_log_reg.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []

for batch in test_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    # print(logits)

    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

xlmr_all_rev_test = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []
for batch in dev_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        b_labels = batch[3].to(device)        

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        loss = loss_fn(logits, b_labels.type_as(logits))
        
    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    # print(logits)
    label_ids = b_labels.cpu().numpy()

    t_labels.append(label_ids)
    t_preds.append(logits)
    

all_dev_labels = np.concatenate(t_labels, axis=0)
all_dev_logits = np.concatenate(t_preds, axis=0)
# Report the final accuracy for this validation run.
# print(all_dev_logits)
avg_val_accuracy = flat_accuracy_single_logit(all_dev_logits, all_dev_labels)
          
print("Dev Accuracy: {0:.4f}".format(avg_val_accuracy))

xlmr_all_rev_dev = 1 / (1 + np.exp(-all_dev_logits))

Majority Vote Ensemble on Dev

In [ ]:
label = []

for i in range(len(xlmr_87_dev)):
  pos = 0
  if xlmr_87_dev[i]>0.5:
    pos = pos + 1
  if xlmr_all_rev_dev[i]>0.5:
    pos = pos + 1
  if xlmr_xlwic_dev[i]>0.5:
    pos = pos + 1
  if mbert_en_no_rev_dev[i]>0.5:
    pos = pos + 1
  if mbert_xlwic_dev[i]>0.5:
    pos = pos + 1

  if pos<3:
    label.append('F')
  else:
    label.append('T')
  
i=0
acc=0
for i in range(0,len(label)):

  if tags[i]==label[i]:
    acc= acc+1
acc = acc/len(label)
print(acc)

Majority Vote Ensemble on Test

In [ ]:
test_preds = []

for i in range(len(xlmr_87_test)):
  pos = 0
  if xlmr_87_test[i]>0.5:
    pos = pos + 1
  if xlmr_all_rev_test[i]>0.5:
    pos = pos + 1
  if xlmr_xlwic_test[i]>0.5:
    pos = pos + 1
  if mbert_en_no_rev_test[i]>0.5:
    pos = pos + 1
  if mbert_xlwic_test[i]>0.5:
    pos = pos + 1

  if pos<3:
    test_preds.append('F')
  else:
    test_preds.append('T')

print("TEST PREDS ARE IN 'test_preds'")

In [ ]:
# SAVE TEST PREDS IF U WANT TO

p = pd.Datafram(test_preds, columns=['tag'])
p.to_csv("ENTER PATH")

italicized text##**5.2** English Inference

---



In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',)

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/datasets/sub/dev_en_rev.csv")
train_df.rename(columns={"sentence1":"sent1", "sentence2":"sent2"}, inplace=True)

dev_df = pd.read_csv("/content/drive/MyDrive/datasets/MCL-WiC/evaluationtestset/ENGLISH/test.csv")
dev_df.rename(columns={"sentence1":"sent1", "sentence2":"sent2"}, inplace=True)
tags = train_df['tag'].to_numpy()

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!pip install transformers

In [ ]:
# Adding a signal without spaces
train_sentences_1 =  []
train_sentences_2 = []

dev_sentences_1 =  []
dev_sentences_2 = []

list_train_sentences_1 = list(train_df['sent1'])
list_train_sentences_2 = list(train_df['sent2'])

list_dev_sentences_1 =  list(dev_df['sent1'])
list_dev_sentences_2 = list(dev_df['sent2'])

for i in range(len(list_train_sentences_1)):
  sentence_1 = list_train_sentences_1[i]
  # print(sentence_1)
  s1 = int(train_df['start1'][i])
  e1 = int(train_df['end1'][i])
  sentence_1 = sentence_1[:s1]+'"'+sentence_1[s1:e1]+'"'+ sentence_1[e1:]
  train_sentences_1.append(sentence_1)

  sentence_2 = list_train_sentences_2[i]
  # print(sentence_2)
  s2 = int(train_df['start2'][i])
  e2 = int(train_df['end2'][i])
  sentence_2 = sentence_2[:s2]+'"'+sentence_2[s2:e2]+'"'+ sentence_2[e2:]
  train_sentences_2.append(sentence_2)
  # print((train_sentences_1[i], train_sentences_2[i]))

for i in range(len(list_dev_sentences_1)):
  d_sentence_1 = list_dev_sentences_1[i]
  # print(dev_data_df['start1'][i])
  s1 = int(dev_df['start1'][i])
  e1 = int(dev_df['end1'][i])
  d_sentence_1 = d_sentence_1[:s1]+'"'+d_sentence_1[s1:e1]+'"'+ d_sentence_1[e1:]
  dev_sentences_1.append(d_sentence_1)

  d_sentence_2 = list_dev_sentences_2[i]
  s2 = int(dev_df['start2'][i])
  e2 = int(dev_df['end2'][i])
  d_sentence_2 = d_sentence_2[:s2]+'"'+d_sentence_2[s2:e2]+'"'+ d_sentence_2[e2:]
  dev_sentences_2.append(d_sentence_2)


print((train_sentences_2[:10]))
print((train_sentences_1[:10]))
print(dev_sentences_1[:10])
print(dev_sentences_2[:10])

XLMR_MIX_MCL


In [ ]:
#xlmr_mcl
from transformers import XLMRobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-large')

encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_train['input_ids'][0]))
print(type(encoded_inputs_train['input_ids']))


encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)

train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = SequentialSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import XLMRobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "xlm-roberta-large"
        hidden_states = False
        self.encoder = XLMRobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer

class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)

model_bert.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/Mix/xlmr_mix_no_en_rev/roberta_check_best_dev'))
model_log_reg.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/Mix/xlmr_mix_no_en_rev/log_reg_check_best_dev'))

model_bert.eval()
model_log_reg.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []

for batch in train_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)    

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

xlmr_mix_mcl_rev_dev_logits = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_preds = []
for batch in dev_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        
    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

xlmr_mix_mcl_rev_test_logits = 1 / (1 + np.exp(-all_dev_logits))

XLMR_MCL

In [ ]:
#xlmr_mcl
from transformers import XLMRobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-large')

encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_train['input_ids'][0]))
print(type(encoded_inputs_train['input_ids']))


encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)


dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)

train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = SequentialSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import XLMRobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "xlm-roberta-large"
        hidden_states = False
        self.encoder = XLMRobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer

class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)



model_bert.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/English/xlmr_mcl_rev/xlmr_check_best_dev'))
model_log_reg.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/English/xlmr_mcl_rev/log_reg_check_best_dev'))

model_bert.eval()
model_log_reg.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []

for batch in train_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        
    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    t_preds.append(logits)
    
all_dev_logits = np.concatenate(t_preds, axis=0)

xlmr_mcl_rev_dev_logits = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_preds = []
for batch in dev_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

xlmr_mcl_rev_test_logits = 1 / (1 + np.exp(-all_dev_logits))

Roberta_MCL

In [ ]:
#roberta_large_mcl

from transformers import RobertaTokenizerFast

# Load the BERT tokenizer.
print('Loading roberta tokenizer...')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-large')

encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_train['input_ids'][0]))
print(type(encoded_inputs_train['input_ids']))


encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)


dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)

train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'],
                              encoded_inputs_train['attention_mask'], train_pos)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos)

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = SequentialSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import RobertaModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "roberta-large"
        hidden_states = False
        self.encoder = RobertaModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, attention_mask = attention_mask)

        return last_layer

class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)


model_bert.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/English/roberta_signal_mcl_rev/roberta_check_best_dev'))
model_log_reg.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/English/roberta_signal_mcl_rev/log_reg_check_best_dev'))

model_bert.eval()
model_log_reg.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_preds = []

for batch in train_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        
    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

roberta_signal_mcl_rev_dev_logits = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_preds = []
for batch in dev_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

roberta_signal_mcl_rev_test_logits = 1 / (1 + np.exp(-all_dev_logits))

Roberta_Augmented

In [ ]:
#roberta_large_aug

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)


model_bert.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/English/roberta_signal_aug_rev/roberta_check_best_dev'))
model_log_reg.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/English/roberta_signal_aug_rev/log_reg_check_best_dev'))

model_bert.eval()
model_log_reg.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_preds = []

for batch in train_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)
        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        
    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    t_preds.append(logits)
    
all_dev_logits = np.concatenate(t_preds, axis=0)

roberta_signal_aug_rev_dev_logits = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_preds = []
for batch in dev_dataloader:

    with torch.no_grad():
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_poses = batch[2].to(device)

        last_layer = model_bert(b_input_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)
roberta_signal_aug_rev_test_logits = 1 / (1 + np.exp(-all_dev_logits))

BERT_Large

In [ ]:
#bert large
from transformers import BertTokenizerFast

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizerFast.from_pretrained('bert-large-cased')

encoded_inputs_train = tokenizer(train_sentences_1, train_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_train['input_ids'][0]))
print(type(encoded_inputs_train['input_ids']))


encoded_inputs_dev = tokenizer(dev_sentences_1, dev_sentences_2, padding = True, truncation = True, return_tensors = 'pt', return_offsets_mapping=True)
print(len(encoded_inputs_dev['input_ids'][0]))
print(type(encoded_inputs_dev['input_ids']))

# create wordpeice indices of the words of interest now
# be aware, that due to signal the actual token have their positions offsetted
train_pos_1 = []
train_pos_2 = []

for j in range(len(train_sentences_1)):
  offsets = encoded_inputs_train['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(train_df['start1'][j])
  s2 = int(train_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  train_pos_1.append(pos1)
  train_pos_2.append(pos2)

dev_pos_1 = []
dev_pos_2 = []

for j in range(len(dev_sentences_1)):
  offsets = encoded_inputs_dev['offset_mapping'][j].detach().numpy()
  second_sent = False
  s1 = int(dev_df['start1'][j])
  s2 = int(dev_df['start2'][j])

  for i, offset in enumerate(offsets):
    if i == 0:
      continue #cls
    if offset[1] == 0:
      second_sent = True

    if offset[0] == s1+1 and second_sent == False:
      pos1 = i
    elif offset[0] == s2+1 and second_sent == True:
      pos2 = i
      break
  dev_pos_1.append(pos1)
  dev_pos_2.append(pos2)

train_pos_1 = torch.LongTensor(train_pos_1)
train_pos_2 = torch.LongTensor(train_pos_2)

train_pos = torch.stack((train_pos_1, train_pos_2), dim =1)

dev_pos_1 = torch.LongTensor(dev_pos_1)
dev_pos_2 = torch.LongTensor(dev_pos_2)

dev_pos = torch.stack((dev_pos_1, dev_pos_2), dim=1)

from torch.utils.data import TensorDataset

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(encoded_inputs_train['input_ids'], encoded_inputs_train['token_type_ids'],
                              encoded_inputs_train['attention_mask'], train_pos)

dev_dataset = TensorDataset(encoded_inputs_dev['input_ids'], encoded_inputs_dev['token_type_ids'],
                              encoded_inputs_dev['attention_mask'], dev_pos)


from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = SequentialSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
dev_dataloader = DataLoader(
            dev_dataset, # The validation samples.
            sampler = SequentialSampler(dev_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import BertModel, AdamW, BertConfig
import torch.nn as nn

class BERTi(nn.Module):

    def __init__(self):
        super(BERTi, self).__init__()

        options_name = "bert-large-cased"
        hidden_states = False
        self.encoder = BertModel.from_pretrained(options_name, output_hidden_states = hidden_states, return_dict = False)

    def forward(self, input_ids, token_type_ids, attention_mask):
        last_layer, _ = self.encoder(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)

        return last_layer

class Logistic_Reg(nn.Module):

  def __init__(self):
    super(Logistic_Reg, self).__init__()

    self.fc1 = nn.Linear(2048, 1)

  def forward(self, x):
    x = self.fc1(x)

    return x

model_bert = BERTi().to(device)
model_log_reg = Logistic_Reg().to(device)

model_bert.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/English/bert_large_mcl_rev_dev/model_bert_dev_best'))
model_log_reg.load_state_dict(torch.load('/content/drive/MyDrive/datasets/sub/models/English/bert_large_mcl_rev_dev/model_log_reg_dev_best'))

model_bert.eval()
model_log_reg.eval()


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_labels = []
t_preds = []

for batch in train_dataloader:
    b_input_ids = batch[0].to(device)
    b_token_type_ids = batch[1].to(device)
    b_attention_mask = batch[2].to(device)
    b_poses = batch[3].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        last_layer = model_bert(b_input_ids, b_token_type_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))
        

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    
    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

bert_large_mcl_rev_dev_logits = 1 / (1 + np.exp(-all_dev_logits))


total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

t_preds = []
for batch in dev_dataloader:
    b_input_ids = batch[0].to(device)
    b_token_type_ids = batch[1].to(device)
    b_attention_mask = batch[2].to(device)
    b_poses = batch[3].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        last_layer = model_bert(b_input_ids, b_token_type_ids, b_attention_mask)
        b_poses = b_poses.unsqueeze(-1).repeat(1, 1, 1024)
        gathered_activations = torch.gather(last_layer, 1, b_poses)
        logits = model_log_reg(gathered_activations.view(gathered_activations.size()[0], -1))

    # Move logits and labels to CPU
    logits = logits.cpu().detach().numpy()
    t_preds.append(logits)
    

all_dev_logits = np.concatenate(t_preds, axis=0)

bert_large_mcl_rev_test_logits = 1 / (1 + np.exp(-all_dev_logits))

Majority Ensemble

In [ ]:
#dev
label = []
for i in range(0,len(xlmr_mcl_rev_dev_logits)):
  pos=0
  if xlmr_mcl_rev_dev_logits[i]>0.5:
    pos = pos+1
  if bert_large_mcl_rev_dev_logits[i]>0.5:
    pos = pos+1
  if roberta_signal_mcl_rev_dev_logits[i]>0.5:
    pos = pos+1
  if roberta_signal_aug_rev_dev_logits[i]>0.5:
    pos = pos+1
  if xlmr_mix_mcl_rev_dev_logits[i]>0.5:
    pos = pos+1

  if pos<3:
    label.append('F')
  else:
    label.append('T')
       
acc=0
for i in range(0,len(label)):
  if tags[i]==label[i]:
    acc= acc+1
acc = acc/len(label)
print(acc)

In [ ]:
#test
label = []
for i in range(0,len(xlmr_mcl_rev_test_logits)):
  pos = 0
  if xlmr_mcl_rev_test_logits[i]>0.5:
    pos = pos+1
  if bert_large_mcl_rev_test_logits[i]>0.5:
    pos = pos+1
  if roberta_signal_mcl_rev_test_logits[i]>0.5:
    pos = pos+1
  if roberta_signal_aug_rev_test_logits[i]>0.5:
    pos = pos+1
  if xlmr_mix_mcl_rev_test_logits[i]>0.5:
    pos = pos+1

  if pos<3:
    label.append('F')
  else:
    label.append('T')

dev_df['tag'] = label
dev_df.head()
dev_df.to_csv("/content/drive/My Drive/datasets/sub/predictions/preds_english.csv",index=False)

## 6. Evaluating given labels 

In [ ]:
# ENTER CSV PATHS OF THE FOLLOWING
# OUR PREDS CAN BE DOWLOADED FROM THE LINK GIVEN IN README

In [ ]:
pred_english_path = ""
pred_non_english_path = "" 
label_english_path = ""
label_arabic_path = ""
label_chinese_path = ""
label_russian_path = ""
label_french_path = ""

In [ ]:
pred_english_df = pd.read_csv(pred_english_path)
pred_non_english_df = pd.read_csv(pred_non_english_path)

label_english_df = pd.read_csv(label_english_path)
label_arabic_df = pd.read_csv(label_arabic_path)
label_chinese_df = pd.read_csv(label_chinese_path)
label_russian_df = pd.read_csv(label_russian_path)
label_french_df = pd.read_csv(label_french_path)

In [ ]:
tags_pred_english = pred_english_df['tag'].replace({'F' : 0, 'T' : 1}).to_numpy()
tags_pred_non_english = pred_non_english_df['tag'].replace({'F' : 0, 'T' : 1}).to_numpy()

tags_label_english = label_english_df['tag'].replace({'F' : 0, 'T' : 1}).to_numpy()
tags_label_arabic = label_arabic_df['tag'].replace({'F' : 0, 'T' : 1}).to_numpy()
tags_label_chinese = label_chinese_df['tag'].replace({'F' : 0, 'T' : 1}).to_numpy()
tags_label_russian = label_russian_df['tag'].replace({'F' : 0, 'T' : 1}).to_numpy()
tags_label_french = label_french_df['tag'].replace({'F' : 0, 'T' : 1}).to_numpy()




In [ ]:
# english accuracy

print("Accuracy on English = "+str((tags_label_english == tags_pred_english).sum()/len(tags_label_english)))

print("Accuracy on Arabic = "+str((tags_label_arabic == tags_pred_non_english[400:600]).sum()/len(tags_label_arabic)))

print("Accuracy on Chinese = "+str((tags_label_chinese == tags_pred_non_english[200:400]).sum()/len(tags_label_chinese)))

print("Accuracy on French = "+str((tags_label_french == tags_pred_non_english[600:800]).sum()/len(tags_label_french)))

print("Accuracy on Russian = "+str((tags_label_russian == tags_pred_non_english[:200]).sum()/len(tags_label_russian)))
